In [ ]:
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
homedir = ""
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')


In [ ]:
#| hide
if running_in_colab:
    !pip3 install nbdev
    !pip3 install mteval

In [ ]:
from mteval.dataset import *
from mteval.comet import *
from mteval.util import *

In [ ]:
import ipywidgets as widgets
from IPython.display import display

mtengines_selection = widgets.SelectMultiple(
    options=['aws','deepl','google', 'microsoft','modernmt'],
    value=['aws'],
    description='MT Engines:',
    disabled=False
)

display(mtengines_selection)


In [ ]:
print(mtengines_selection.value)

In [ ]:
import csv
import sys
from pathlib import Path

base_path = homedir+"/mtd_data/"
dataset_fname = "datasets.csv"
metrics_fname = "comet.csv"
normalize_quotes = False

comet_scorer = cometscoring()
with open(base_path+dataset_fname,'r') as datasets_fh:
    set_reader = csv.reader(datasets_fh)
    for (source_language_code,target_language_code,test_set_name,test_date,domain) in set_reader:
        for mtengine in mtengines_selection.value:
            source_lines = []
            reference_lines = []
            source_lines, reference_lines = download_read_set(base_path,source_language_code,target_language_code,test_set_name)
            target_lines = []
            try:
                target_lines = get_translated_test_set(base_path,source_language_code,target_language_code,mtengine,test_set_name,test_date,domain)
            except FileNotFoundError:
                print("Translated file not found",file=sys.stderr)
                continue
            if normalize_quotes == True:
                utils = util()
                source_lines = [utils.normalize_quotes(source_line) for source_line in source_lines]
                reference_lines = [utils.normalize_quotes(reference_line) for reference_line in reference_lines]
                target_lines = [utils.normalize_quotes(target_line) for target_line in target_lines]
            print("COMET benchmarking {}-{} Testset: {} Date: {} MT engine: {} Domain: {}".format(source_language_code,target_language_code,test_set_name,test_date,mtengine,domain))
            comet_scorer.measure_record_comet(source_lines,target_lines,reference_lines,source_language_code,target_language_code,test_set_name,test_date,mtengine,base_path,metrics_fname,domain)
